In [2]:
import pandas as pd
import numpy as np
import os
import warnings

In [22]:
# Processing FRED Data - Merging all datasets together at the monthly level

# Cheat Sheet for FRED Files
# FREDFUNDS      =  "The federal funds rate is the interest rate at which depository institutions trade federal funds (balances held at Federal Reserve Banks) with each other overnight." -> The standard metric used when people the government is changing the interest rate.
    # Monthly
    # Starts in July 1954
# RSAHORUSQ156S  =  "The homeownership rate is the proportion of households that is owner-occupied." -> A percentage
    # Quarterly
    # Starts in Quarter 1 of 1980
# MORTGAGE30US   =  "30-Year Fixed Rate Mortgage Average in the United States" -> Average mortgage rate for 30 year fixed
    # Weekly, Ending on Thursday
    # Starts in April 2, 1971
# CPIAUCSL       =  "The Consumer Price Index for All Urban Consumers" -> Can be used to adjust for inflation
    # Monthly
    # Starts in January 1947

# File list
FREDdir = '../data/FRED/'
file_list = os.listdir(FREDdir)

# Loop through file list to get fully qualified paths for reading in pandas and put them in a dictionary with the file name (without extension) as the key
path = {}
for file in file_list:
    path[file.split('.')[0]] = os.path.join(FREDdir,file)
#print(path)

# After a brief manual browsing of the data, I found that the information actually begins on row 11 which is 10 on a 0 index
# So, we will put that row as the headers
# Open all the files in pandas and store them in a dictionary with the key as the filename
excel_files = {}
for key, value in path.items():
    if 'ipynb' in value:
        continue
    else:
        excel_files[key] = pd.read_excel(value, header=10)

# Check for nulls        
RSAHORUSQ156S = excel_files['RSAHORUSQ156S'][excel_files['RSAHORUSQ156S']['RSAHORUSQ156S'].isnull()].count()
FEDFUNDS = excel_files['FEDFUNDS'][excel_files['FEDFUNDS']['FEDFUNDS'].isnull()].count()
MORTGAGE30US = excel_files['MORTGAGE30US'][excel_files['MORTGAGE30US']['MORTGAGE30US'].isnull()].count()
CPIAUCSL = excel_files['CPIAUCSL'][excel_files['CPIAUCSL']['CPIAUCSL'].isnull()].count()

#If sum > 1 then there are nulls in the data
if RSAHORUSQ156S[1] > 0:
    warnings.warn("Source dataset RSAHORUSQ156S has nulls and will need cleaned")
if FEDFUNDS[1] > 0:
    warnings.warn("Source dataset FEDFUNDS has nulls and will need cleaned")
if MORTGAGE30US[1] > 0:
    warnings.warn("Source dataset MORTGAGE30US has nulls and will need cleaned")
if CPIAUCSL[1] > 0:
    warnings.warn("Source dataset CPIAUCSL has nulls and will need cleaned")
#print(sum([RSAHORUSQ156S[1],FEDFUNDS[1],MORTGAGE30US[1],CPIAUCSL[1]])) # Sum = 0

# Convert date to quarter for FEDFUNDS
excel_files['FEDFUNDS']['date'] = pd.to_datetime(excel_files['FEDFUNDS']['observation_date']) # create date column as date time data type
excel_files['FEDFUNDS']['quarter'] =  pd.PeriodIndex(excel_files['FEDFUNDS']['date'], freq='Q') # converts to quarter
excel_files['FEDFUNDS']['month'] =  pd.PeriodIndex(excel_files['FEDFUNDS']['date'], freq='M') # converts to month
#excel_files['FEDFUNDS'] = excel_files['FEDFUNDS'].drop(columns=['date','observation_date']) # drop other date columns
excel_files['FEDFUNDS'] = excel_files['FEDFUNDS'].drop(columns=['date']) 
#excel_files['FEDFUNDS'] = excel_files['FEDFUNDS'].groupby('quarter').mean().reset_index()
#excel_files['FEDFUNDS'].head()

# Change RSAHORUSQ156S formatting to month in order to merge with FEDFUNDS
excel_files['RSAHORUSQ156S']['date'] = pd.to_datetime(excel_files['RSAHORUSQ156S']['observation_date'])
excel_files['RSAHORUSQ156S']['quarter'] =  pd.PeriodIndex(excel_files['RSAHORUSQ156S']['date'], freq='Q')
excel_files['RSAHORUSQ156S'] = excel_files['RSAHORUSQ156S'].drop(columns=['date','observation_date'])
#excel_files['RSAHORUSQ156S'].head()

# Merge the datasets - FEDFUNDS & RSAHORUSQ156S
FEDFUND_RSAH = pd.merge(excel_files['FEDFUNDS'], excel_files['RSAHORUSQ156S'], how = 'outer', on = 'quarter') # outer join in order to keep all data
FEDFUND_RSAH = FEDFUND_RSAH[['quarter', 'month', 'observation_date','FEDFUNDS','RSAHORUSQ156S']]

# Change MORTGAGE30US formatting to month in order to merge with FEDFUNDS
excel_files['MORTGAGE30US']['date'] = pd.to_datetime(excel_files['MORTGAGE30US']['observation_date'])
excel_files['MORTGAGE30US']['month'] =  pd.PeriodIndex(excel_files['MORTGAGE30US']['date'], freq='M')
excel_files['MORTGAGE30US'] = excel_files['MORTGAGE30US'].drop(columns=['date','observation_date'])
#excel_files['MORTGAGE30US'].head()

# Merge the datasets - (FEDFUNDS & RSAHORUSQ156S) & MORTGAGE30US
FEDFUND_RSAH_MORT = pd.merge(FEDFUND_RSAH, excel_files['MORTGAGE30US'], how = 'outer', on = 'month')
FEDFUND_RSAH_MORT = FEDFUND_RSAH_MORT[['quarter', 'month','FEDFUNDS','RSAHORUSQ156S', 'MORTGAGE30US']]
#FEDFUND_RSAH_MORT.head(400)

# Change CPIAUCSL formatting to month in order to merge with FEDFUNDS
excel_files['CPIAUCSL']['date'] = pd.to_datetime(excel_files['CPIAUCSL']['observation_date'])
excel_files['CPIAUCSL']['month'] =  pd.PeriodIndex(excel_files['CPIAUCSL']['date'], freq='M')
excel_files['CPIAUCSL'] = excel_files['CPIAUCSL'].drop(columns=['date','observation_date'])
#excel_files['CPIAUCSL'].head()

# Merge the datasets - (FEDFUNDS & RSAHORUSQ156S & MORTGAGE30US) & CPIAUCSL
FEDFUND_RSAH_MORT_CPI = pd.merge(FEDFUND_RSAH_MORT, excel_files['CPIAUCSL'], how = 'outer', on = 'month')
FEDFUND_RSAH_MORT_CPI = FEDFUND_RSAH_MORT_CPI[['quarter', 'month','FEDFUNDS','RSAHORUSQ156S', 'MORTGAGE30US', 'CPIAUCSL']]
FEDFUND_RSAH_MORT_CPI.head(400)

# FRED Datasets are Merged!

,quarter,month,FEDFUNDS,RSAHORUSQ156S,MORTGAGE30US,CPIAUCSL
0,1954Q3,1954-07,0.80,NaN,NaN,26.86
1,1954Q3,1954-08,1.22,NaN,NaN,26.85
2,1954Q3,1954-09,1.07,NaN,NaN,26.81
3,1954Q4,1954-10,0.85,NaN,NaN,26.72
4,1954Q4,1954-11,0.83,NaN,NaN,26.78
...,...,...,...,...,...,...
395,1974Q4,1974-12,8.53,NaN,9.58,51.90
396,1974Q4,1974-12,8.53,NaN,9.56,51.90
397,1975Q1,1975-01,7.13,NaN,9.60,52.30
398,1975Q1,1975-01,7.13,NaN,9.49,52.30
